In [ ]:
import openai
import os
from pprint import pprint
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores.azuresearch import AzureSearch
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI

# read local .env file
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) 

# setup azure openai api
openai.api_type = "azure"
openai.api_base = os.getenv("AZURE_OAI_ENDPOINT")
# openai.api_version = "2023-03-15-preview"
openai.api_version = "2023-05-15"
openai.api_key  = os.getenv('AZURE_OAI_KEY')

llm_name = "gpt-3.5-turbo"
deployment_name_gpt = os.getenv("AZURE_OAI_MODEL_GPT_3")
deployment_name_ada = os.getenv("AZURE_OAI_MODEL_ADA")

vector_store_address: str = os.getenv("AZURE_SEARCH_ENDPOINT")
vector_store_password: str = os.getenv("AZURE_SEARCH_ADMIN_KEY")

os.environ["OPENAI_API_TYPE"] = openai.api_type
os.environ["OPENAI_API_BASE"] = openai.api_base
os.environ["OPENAI_API_KEY"] = openai.api_key
os.environ["OPENAI_API_VERSION"] = openai.api_version

In [ ]:
llm = AzureChatOpenAI(deployment_name=deployment_name_gpt)

In [ ]:
embeddings: OpenAIEmbeddings = OpenAIEmbeddings(deployment=deployment_name_ada, chunk_size=1)
index_name: str = "facts-about-snakes"
azure_search_store: AzureSearch = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    semantic_configuration_name="default",
    embedding_function=embeddings.embed_query,
)

In [ ]:
azure_search_retreiver = azure_search_store.as_retriever(search_kwargs={"k":4})

### chains

In [ ]:
from langchain.chains import RetrievalQA
acs_qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=azure_search_retreiver)

In [ ]:
pprint(acs_qa.run("what are the snakes covered in the document?"))

### Tools

In [ ]:
from langchain.agents import Tool

description=(
    "Useful FAQ about Snakes in North Carolina. "
    "You should ALWAYS use this tool whenever you need to answer a question about Snakes, "
)

acs_tool = Tool(name="acs_search", description=description, func=acs_qa.run)


In [ ]:
pprint(acs_tool)

### Create Agents from tools

In [ ]:
from langchain.agents import AgentType
from langchain.memory import ConversationBufferMemory
from langchain.agents import initialize_agent

In [ ]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
tools = [acs_tool]
agent_chain = initialize_agent(tools, llm, agent=AgentType.CHAT_CONVERSATIONAL_REACT_DESCRIPTION, verbose=True, memory=memory)

### Test refined tools

In [ ]:
agent_chain.run("What are the venomous snakes in North Carolina?")

In [ ]:
agent_chain.run("give me some facts about Eastern Coral Snake?")

In [ ]:
memory.load_memory_variables({})

### Custom Tool

In [ ]:
from typing import Optional, Type
from langchain.tools import BaseTool

from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

In [ ]:
class CustomSearchTool(BaseTool):
    name = "custom_search"
    description = "useful for when you need to answer search from azure cognitive seach"

    def _run(
        self, query: str, run_manager: Optional[CallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool."""
        return azure_search_store.similarity_search(
            query=query, 
            k=4,
            search_type="semantic_hybrid"
        )

    async def _arun(
        self, query: str, run_manager: Optional[AsyncCallbackManagerForToolRun] = None
    ) -> str:
        """Use the tool asynchronously."""
        raise NotImplementedError("custom_search does not support async")


In [110]:
# from langchain.agents import tool

# @tool
# def acs_search_semantic(query: str):
#     """Returns the length of a word."""
#     return azure_search_store.similarity_search(
#             query=query, 
#             k=4,
#             search_type="semantic_hybrid"
#         )

In [114]:
acs_tools = [CustomSearchTool()]
# acs_tools = [acs_search_semantic]
agent = initialize_agent(
    acs_tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [115]:
agent.run("Give me facts about Eastern Coral Snake? ")



> Entering new AgentExecutor chain...
I should use custom_search to find information about the Eastern Coral Snake.
Action: custom_search
Action Input: "Eastern Coral Snake facts"
Observation: [Document(page_content='Eastern Coral Snake. The Copperhead is the most common \nand widespread venomous snake in North Carolina, occur-\nring in both rural and urban environments. Four of the six \nare protected species in North Carolina, and as such, should \nnot be handled or disturbed: the Timber and Pigmy Rattle-\nsnakes are both Species of Special Concern, while the \nEastern Diamondback Rattlesnake and Eastern Coral Snake \nare both Endangered.  \nWhy do snakes shed their skins?\nAs a snake grows, its body increas es in size, b ut its skin \ndoes not grow with it. Therefore, as they grow, snakes \nmust replace their sma ller skins with lar ger ones. The act \nof shedding old s kin also removes  any parasites t hat may \nhave been presen t on their skin. Snak es shed their  skins \nin a m

'The Eastern Coral Snake is a venomous snake that is endangered and a protected species in North Carolina. It is found in both rural and urban environments.'